In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px

# Carregar a base limpa salva no módulo anterior
# Usamos delimiter=',' pois o pandas salva o CSV por padrão com vírgulas
df = pd.read_csv("churn_telecon_limpo.csv", delimiter=',')

print("Base limpa carregada e pronta para Análise Exploratória (AED).")
print(df.info())

# A) df.describe() para Estatísticas Descritivas
print("A) Estatísticas Descritivas (describe) de Variáveis Quantitativas:")
print(df.describe().T.to_markdown())

# C) Plot de 4 Variáveis (Univariada)

# Plot 1: Distribuição do Tempo como Cliente (Quantitativa)
plt.figure(figsize=(8, 5))
sns.histplot(df['tempo_como_cliente'], kde=True, bins=30, color='skyblue')
plt.title('Distribuição do Tempo como Cliente (Meses)')
plt.show()

# Plot 2: Distribuição do Pagamento Mensal (Quantitativa)
plt.figure(figsize=(8, 5))
sns.histplot(df['pagamento_mensal'], kde=True, bins=30, color='lightcoral')
plt.title('Distribuição do Pagamento Mensal')
plt.show()

# Plot 3: Distribuição do Serviço de Internet (Categórica)
plt.figure(figsize=(8, 5))
sns.countplot(y=df['servico_internet'], order = df['servico_internet'].value_counts().index, palette='viridis')
plt.title('Contagem por Tipo de Serviço de Internet')
plt.show()

# D) Verificação do balanceamento da variável preditora (Churn)
plt.figure(figsize=(6, 4))
sns.countplot(x='churn', data=df, palette='Pastel1')
plt.title('Distribuição da Variável Churn (Balanceamento)')
plt.show()
print("\nD) Distribuição da Variável Churn:")
print(df['churn'].value_counts(normalize=True).round(2).to_markdown())

### Insights da Análise Univariada

#### **A) Estatísticas Descritivas e B) Identificação de Outliers**

* **Pagamento Mensal (`pagamento_mensal`):** A média (64.75) e a mediana (70.35) são próximas. A distribuição está concentrada nos valores mais altos (acima de 70), mas também possui um pico de clientes com pagamentos muito baixos (R$ 20-30), provavelmente aqueles que contratam apenas telefone.
* **Total Pago (`total_pago`):** O valor máximo (8672.45) é muito superior ao 75º percentil (Q3: 3788.85), indicando uma **forte assimetria positiva**.
* **B) Outliers:** Os **outliers** mais evidentes aparecem na coluna **`total_pago`**. Esses valores extremos representam clientes que acumularam um alto valor ao longo de anos de contrato. A análise posterior definirá se serão mantidos ou removidos.

#### **C) Insights dos Gráficos**

1.  **Tempo como Cliente (`tempo_como_cliente`):** O gráfico sugere que o risco de Churn é mais alto para **clientes novos** (Tenure baixo) e **clientes antigos** (Tenure alto) que não saíram estão bem fidelizados.
2.  **Pagamento Mensal (`pagamento_mensal`):** Há dois grupos distintos: clientes de baixo pagamento (apenas telefone/DSL básico) e clientes de alto pagamento (pacotes completos de Fibra Ótica e serviços adicionais).
3.  **Serviço de Internet (`servico_internet`):** A **Fibra Ótica** é o serviço dominante (mais contratado), o que será importante na análise bivariada, pois pode estar associado a problemas de qualidade e, consequentemente, a uma alta taxa de Churn.
4.  **D) Balanceamento da Variável Churn:**
    * A base está **desbalanceada**. Cerca de **73%** dos clientes **Não** deram Churn (`no`), enquanto **27%** **Sim** deram Churn (`yes`).
    * **Implicação:** Na etapa de Modelagem, este desbalanceamento exigirá técnicas específicas (como SMOTE ou ajuste de pesos) para que o modelo consiga prever a minoria (`yes`) com precisão.

---

### Célula 3: 3) Identificação e Tratamento de Outliers

*Usaremos o `total_pago`, o principal candidato a ter *outliers**, para aplicar a regra de domínio.*

```python
# seu código aqui

# 1. Visualização de Outliers em total_pago (Box Plot)
plt.figure(figsize=(5, 6))
sns.boxplot(y=df['total_pago'], color='orange')
plt.title('Box Plot da Variável Total Pago')
plt.show()

# 2. Análise e Decisão:
# Manter ou Remover? Decisão é MANTER.
# Justificativa: Os valores altos na coluna `total_pago` são resultado da multiplicação do `pagamento_mensal` pelo `tempo_como_cliente`. 
# Estes valores representam clientes de alto valor acumulado, que são **legítimos** e não erros de medição. 
# Remover estes outliers resultaria na perda de informações sobre clientes mais rentáveis e antigos, o que é crucial para o modelo preditivo.

# --- A) 5 Questões Bivariadas vs. CHURN ---

# Pergunta 1: Qual o impacto do Tipo de Contrato na taxa de Churn?
fig1 = px.bar(
    df.groupby('tipo_contrato')['churn'].value_counts(normalize=True).mul(100).rename('Porcentagem').reset_index(),
    x='tipo_contrato', y='Porcentagem', color='churn', barmode='group',
    title='1. Taxa de Churn por Tipo de Contrato',
    labels={'tipo_contrato': 'Tipo de Contrato', 'Porcentagem': 'Taxa de Churn (%)'}
)
fig1.show()

# Pergunta 2: O tipo de Serviço de Internet influencia a taxa de Churn?
fig2 = px.bar(
    df.groupby('servico_internet')['churn'].value_counts(normalize=True).mul(100).rename('Porcentagem').reset_index(),
    x='servico_internet', y='Porcentagem', color='churn', barmode='group',
    title='2. Taxa de Churn por Serviço de Internet',
    labels={'servico_internet': 'Serviço de Internet', 'Porcentagem': 'Taxa de Churn (%)'}
)
fig2.show()

# Pergunta 3: Clientes que dão Churn são novos ou antigos (Tenure)?
fig3 = px.box(
    df, x='churn', y='tempo_como_cliente', color='churn',
    title='3. Churn vs. Distribuição do Tempo como Cliente (Tenure)',
    labels={'tempo_como_cliente': 'Tempo como Cliente (Meses)'}
)
fig3.show()

# Pergunta 4: A posse de Suporte Técnico afeta o Churn?
fig4 = px.bar(
    df.groupby('suporte_tecnico')['churn'].value_counts(normalize=True).mul(100).rename('Porcentagem').reset_index(),
    x='suporte_tecnico', y='Porcentagem', color='churn', barmode='group',
    title='4. Taxa de Churn por Suporte Técnico',
    labels={'suporte_tecnico': 'Tem Suporte Técnico?'}
)
fig4.show()

# Pergunta 5: O valor do Pagamento Mensal é maior para quem dá Churn?
fig5 = px.box(
    df, x='churn', y='pagamento_mensal', color='churn',
    title='5. Churn vs. Distribuição do Pagamento Mensal',
    labels={'pagamento_mensal': 'Pagamento Mensal'}
)
fig5.show()

### Insights da Análise Bivariada

#### **A) Interpretação dos Gráficos**

1.  **Taxa de Churn por Tipo de Contrato:**
    * Clientes com contrato **"month-to-month"** (mês a mês) têm uma taxa de Churn superior a 40%. Esta é a variável de maior risco, indicando falta de fidelidade.
2.  **Taxa de Churn por Serviço de Internet:**
    * Clientes de **Fibra Ótica** apresentam a maior taxa de Churn (quase 42%), enquanto clientes sem serviço de internet (No internet service) ou com DSL têm taxas muito mais baixas. Isso aponta para problemas de satisfação com a Fibra.
3.  **Churn vs. Tempo como Cliente (Tenure):**
    * A mediana do tempo de contrato para clientes que dão Churn (`yes`) é de apenas **10 meses**. A mediana para clientes que permanecem (`no`) é de **38 meses**. Isso confirma que a maior taxa de Churn ocorre no primeiro ano de contrato.
4.  **Taxa de Churn por Suporte Técnico:**
    * A ausência de Suporte Técnico (variável `no`) está diretamente relacionada a uma taxa de Churn mais alta. Clientes sem suporte ficam mais insatisfeitos e saem.
5.  **Churn vs. Pagamento Mensal:**
    * Clientes que saem (`yes`) têm um valor de **Pagamento Mensal mediano mais alto** do que aqueles que ficam (`no`). Isso é crítico, pois indica que a empresa está perdendo seus clientes de maior valor.

#### **B) Variáveis Mais Importantes para o Projeto (Relacionadas ao Churn)**

As variáveis com maior poder preditivo (que devem ser o foco na modelagem) são:

1.  **`tipo_contrato`:** O fator de fidelidade mais forte.
2.  **`tempo_como_cliente`:** Indica o período de maior risco (clientes novos).
3.  **`servico_internet`:** A alta taxa de Churn da Fibra Ótica aponta um problema de qualidade que é um preditor direto.
4.  **`pagamento_mensal`:** O Churn é maior entre clientes de alto pagamento (alto valor), crucial para o foco do negócio.
5.  **`suporte_tecnico`:** A ausência ou presença deste serviço impacta diretamente a retenção.